In [1]:
#imports
from cnn import *
from timeit import timeit,Timer
from GameController import *
#import itertools
import sys
#import objgraph
import os
import psutil
process = psutil.Process(os.getpid())

In [2]:
#Modified Functions
#infer_action
#get_seq_y
#train_network

#Relevant Functions
def infer_action(seq):
    action = tf.get_default_graph().get_tensor_by_name("action:0")
    q_compute = tf.get_default_graph().get_tensor_by_name("Qnext_val:0")
    outputs = tf.get_default_graph().get_tensor_by_name("dense_layers/FC2/act2/Maximum:0")
    x1,y,x2,next_state_bool,Qnext= get_place_holders()
    #return sess.run([action],{x1: seq,x2: np.random.rand(1,110,84,4),next_state_bool: False})
    a,q = sess.run([action,q_compute],{x1: seq})
    return a

def send_action_to_game_controller(game,a):
    if (a == 0):
        game.move_mouse_up()
    elif (a == 1):
        game.move_mouse_down()
    elif (a == 2):
        game.move_mouse_left()
    else:
        game.move_mouse_right()
    
    r = game.get_reward()
    frames,bval = get_4_frames(game)
    return r,frames,bval

def random_minibatch_sample(batchsize):
    global exp
    
    line_N = np.random.randint(0,len(exp),size=batchsize)
    return np.asarray([exp[i] for i in line_N]).transpose()
    
def get_seq_y(seq,batch_size,gamma):
    q_compute = tf.get_default_graph().get_tensor_by_name("Qnext_val:0")
    x1,y,x2,next_state_bool,Qnext = get_place_holders()
    
    imgs_2 = [((1.0/256)*np.array(seq[3,i])).astype(np.float16) for i in range(0,batch_size)]
    imgs_1 = [((1.0/256)*np.array(seq[0,i])).astype(np.float16) for i in range(0,batch_size)]

    
    #dummy = np.random.rand(1,110,84,4)
    #q_vals = sess.run([q_compute],{x1: dummy,x2: imgs_2,next_state_bool: True})
    q_vals = sess.run([q_compute],{x1: imgs_2})
    r_vals = [seq[2,i] for i in range(0,batch_size)]
    
    q_vals = np.array(q_vals)
    r_vals = np.array(r_vals)
    y = (r_vals+(gamma*q_vals)).reshape(len(r_vals),1)
    return y,np.squeeze(np.array(imgs_1))
    
def store_exp(seq):
    global exp
    img_1 = seq[0]
    img_2 = seq[3]
    img_1 = (256*img_1).round().astype(np.uint8)
    img_2 = (256*img_2).round().astype(np.uint8)
    seq[0] = img_1.tolist()
    seq[3] = img_2.tolist()
    exp.append(seq)
    return

def train_network(batch_size,gamma):
    #Required tensorflow variables and operations
    global writer,summ,it
    x1,y,x2,next_state_bool,Qnext = get_place_holders()
    train = tf.get_default_graph().get_operation_by_name("train/trainer")
    seq = random_minibatch_sample(batch_size)
    #Getting y values and the corresponding training images
    y_vals,images = get_seq_y(seq,batch_size,gamma)
    images = np.squeeze(images)
    #dummy = np.random.rand(1,110,84,4)
    #t,s = sess.run([train,summ],{x1: images,x2: dummy,Qnext: y_vals,next_state_bool:False})
    t,s = sess.run([train,summ],{x1: images, Qnext: y_vals})
    writer.add_summary(s,it)
    it = it+1
    return

def get_4_frames(game):
    imgs = [take_shot(game) for i in range(0,4)]
    bval = [game.get_screen_number(i) for i in imgs]
    #imgs = [img_standardize(i) for i in imgs]
    imgs = img_standardize(imgs)
    imgs = np.rollaxis(imgs,0,3)
    if max(bval) == 3:
        return imgs,True
    else:
        return imgs,False

In [3]:
#Train/Play Fucnction
def run(game,greed,M,batch_size,gamma):
    global exp
    for i in range(0,M):  #New play
        wait_for(1)
        game.click_to_play()
        seq = []
        p = 0
        while game.get_screen_number(take_shot(game)) is not 3: #for j in range(0,T): #While play active
            frames,test = get_4_frames(game)
            if (test):
                break
            seq.append(frames)
            if np.random.random_sample(1) <= greed:
                a = np.random.randint(0,4)
            else:
                a = infer_action([seq[0]])
            r,frames,test = send_action_to_game_controller(game,a)
            if (test):
                break
            seq.append(a)
            seq.append(r)
            seq.append(frames)
            store_exp(seq)
            seq = []
            if (len(exp) > 10):
                train_network(batch_size,gamma)
        game.release_click()
        wait_for(.3)
        game.click_replay()
        print("Iteration: ",i)
        #print("size: ",len(exp))
        #print("size bytes: ",sys.getsizeof(exp))
        #print("mem: ",(process.memory_info().rss)/1000000)
        
def play_game(game,M):
    for i in range(0,M):
        wait_for(1)
        game.click_to_play()
        while game.get_screen_number(take_shot(game)) is not 3:
            frames,test = get_4_frames(game)
            if (test):
                break
            a = infer_action([frames])
            r,frames,test = send_action_to_game_controller(game,a)
            if (test):
                break
        game.release_click()
        wait_for(.3)
        game.click_replay()
        print("Play Iteration: ",i)

In [4]:
conv_k_size = [8,4]
conv_stride = [4,2]
conv = [0,16,32]
fclyr = [0,125,4]
conv_count = len(conv)
fc_count = len(fclyr)
learning_rate = 1e-4
batch_size = 10
LOGDIR = r"c:\Users\Vishnu\Documents\EngProj\SSPlayer\log"
sess,writer,summ,place_holders= create_model(learning_rate,batch_size,conv_count,fc_count,conv,fclyr,conv_k_size,conv_stride,LOGDIR)

writer.add_graph(sess.graph)

sdevconv1:  0.08838834764831845
sdevconv2:  0.08838834764831845
sdevFC1:  0.011180339887498949
sdevFC2:  0.06324555320336758


In [5]:
it = 0
exp = []
app_dir = r"C:\Users\Vishnu\Documents\EngProj\SSPlayer\Release.win32\ShapeScape.exe"
if 1:
    if __name__ == "__main__":
        game = SSPlayer(app_dir,1)
        wait_for(1)
        game.click_play()
        run(game,.7,25,2,.9)
        print("Exp size: ", sys.getsizeof(exp)/1000.0)
        print("Now Playing")
        play_game(game,10)
        sess.close()
        game.kill()
        #sess.close()
        #t_img = multiprocessing.Queue()
        #ev = multiprocessing.Event()
        #pp = game.processing_crop
        #p = multiprocessing.Process(target=multi_add_training_images,args=[t_img,ev,pp])

        #print(Timer(lambda: Run(game,p,ev,t_img)).timeit(number=1))def save_imgs(exp):

c:\users\vishnu\envs\rl\lib\site-packages\pywinauto\application.py:1032: UserWarning: 32-bit application should be automated using 32-bit Python (you use 64-bit Python)
  UserWarning)


Iteration:  0
Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Exp size:  1.072
Now Playing
Play Iteration:  0
Play Iteration:  1
Play Iteration:  2
Play Iteration:  3
Play Iteration:  4
Play Iteration:  5
Play Iteration:  6
Play Iteration:  7
Play Iteration:  8
Play Iteration:  9


In [6]:
#Other Functions
def save_imgs(exp):
    img_1 = [(250*((1.0/256)*np.array(exp[i][0]))).astype(np.uint8) for i in range(0,len(exp))]
    img_2 = [(250*((1.0/256)*np.array(exp[i][3]))).astype(np.uint8) for i in range(0,len(exp))]
    
    print(np.shape(img_1))
    print(np.shape(img_2))
    
    imgs= []
    for i in img_1:
        imgs.append(i[:,:,0])
        imgs.append(i[:,:,1])
        imgs.append(i[:,:,2])
        imgs.append(i[:,:,3])
        
    
    for i in img_2:
        imgs.append(i[:,:,0])
        imgs.append(i[:,:,1])
        imgs.append(i[:,:,2])
        imgs.append(i[:,:,3])
        
    
    print(np.shape(imgs))
    print(sys.getsizeof(imgs)/1000)
    for i in range(0,len(imgs)):
        Image.fromarray(imgs[i]).save(r"test\frame"+str(i)+".png")
    return
        
def test_seq(game,batch_size):
    seq = []
    for i in range(0,batch_size):
        frames,t = get_4_frames(game,game.processing_crop)
        seq.append(frames)
        seq.append(np.random.randint(0,4))
        seq.append(np.random.random_sample(1))
        frames,t = get_4_frames(game,game.processing_crop)
        seq.append(frames)
    return seq

def test():
    x1,y,x2,next_state_bool,Qnext = get_place_holders()
    train = tf.get_default_graph().get_operation_by_name("train/trainer")
    dummy = np.random.rand(25,110,84,4)
    y_vals = np.random.rand(25,1)
    sess.run([train],{x1: dummy,x2: dummy,Qnext: y_vals,next_state_bool:False})
    return

if False:
    print(Timer(lambda: test()).timeit(number=1))